In [ ]:
import random
import pandas as pd

class TabuSearch:
    def __init__(self, data_path, max_solutions, max_tabu_size, max_iterations):
        self.data = pd.read_excel(data_path, engine='openpyxl')
        self.distance_matrix = self.create_distance_matrix()
        self.num_schools = len(self.distance_matrix.columns)
        self.num_teachers = len(self.distance_matrix.index)
        self.tabu_list = []
        self.max_tabu_size = max_tabu_size
        self.max_iterations = max_iterations
        self.max_solutions = max_solutions

    def create_distance_matrix(self):
        data = self.data.groupby(["NAMA_GURU", "NAMA_SEKOLAH"])["JARAK"].sum().reset_index()
        distance_matrix = data.pivot(index="NAMA_GURU", columns="NAMA_SEKOLAH", values="JARAK").fillna(0)
        return distance_matrix

    def calculate_total_distance(self, assignment):
        total_distance = 0
        for teacher, school in assignment.items():
            total_distance += self.distance_matrix.loc[teacher, school]
        return total_distance

    def get_initial_solution(self):
        solution = {}
        schools = list(self.distance_matrix.columns)
        teachers = list(self.distance_matrix.index)
        random.shuffle(teachers)
        for school in schools:
            for _ in range(6):  # Each school has 6 classes
                if not teachers:
                    break
                solution[teachers.pop(0)] = school
        return solution

    def tabu_search(self):
        best_solution = self.get_initial_solution()
        best_distance = self.calculate_total_distance(best_solution)

        for iteration in range(self.max_iterations):
            print(f"Iterasi ke-{iteration}")
            print("Beberapa Rute Terlebih Dahulu:")
            sorted_neighbors = sorted(self.get_neighbors(best_solution), key=lambda x: self.calculate_total_distance(x))
            for i, neighbor in enumerate(sorted_neighbors[:3], 1):
                print(f"Rute {i}: {list(neighbor.keys())} Jarak: {self.calculate_total_distance(neighbor)}")

            neighbors = self.get_neighbors(best_solution)
            neighbors.sort(key=lambda x: self.calculate_total_distance(x))
            best_neighbor = neighbors[0]
            best_neighbor_distance = self.calculate_total_distance(best_neighbor)

            if best_neighbor_distance < best_distance:
                best_solution = best_neighbor
                best_distance = best_neighbor_distance
                self.tabu_list.append(best_neighbor)
                print(f"Rute baru {list(best_solution.keys())}")
                print(f"Jarak {best_distance}")
                print(f"Tabu List {self.tabu_list}")
            else:
                print("Tidak ada perbaikan, mengulangi iterasi.")
                print(f"Tabu List {self.tabu_list}")

    def get_neighbors(self, solution):
        neighbors = []
        for teacher, school in solution.items():
            for new_school in self.distance_matrix.columns:
                if new_school != school:
                    neighbor = solution.copy()
                    neighbor[teacher] = new_school
                    if neighbor not in self.tabu_list:
                        neighbors.append(neighbor)
        return neighbors

# Input parameter dari pengguna
data_path = input("Masukkan nama file Excel dengan data guru dan sekolah: ")
max_solutions = int(input("Masukkan jumlah solusi per iterasi (4, 6, 8, 10): "))
max_tabu_size = int(input("Masukkan panjang tabu list (2, 3, 4): "))
max_iterations = int(input("Masukkan jumlah iterasi (10, 20, 30, 40, 50, 100, 1000): "))

# Menjalankan Tabu Search dengan parameter yang diinputkan pengguna
tabu_search = TabuSearch(data_path, max_solutions, max_tabu_size, max_iterations)
tabu_search.tabu_search()
